## 1.Layer and block

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

#MLP
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X=torch.rand(2,20)

#自定义Block(深度神经网络中比层中单元或层更加宏观的单位)
class MLP(nn.Module): #继承nn.Module类
    #定义构造函数
    def __init__(self):
        super().__init__() #调用父类Module的构造函数来执行必要的初始化。 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out=nn.Linear(256,10) #输出层
    #定义前向传播函数，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))#使用ReLU的函数版本，定义在nn.functional模块中。

net=MLP() #实例化多层感知机层
net(X)

#Sequential类模拟
class MySequential(nn.Module): #继承nn.Module类
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):#enumerate(args)返回索引-实例的元组，args是`Module`子类的一个实例
            self._modules[str(idx)]=module#将module保存在'Module'类的成员变量`_modules` 中。`_module`的类型是OrderedDict
    
    #MySequential的前向传播函数
    def forward(self,X):
        for block in self._modules.values():# OrderedDict保证了按照成员添加的顺序遍历Module实例
            X=block(X) #按顺序对X执行module 
        return X
    
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10)) #实例化MySequential，用法与Sequential类相同
net(X)

#在前向传播代码中执行函数
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)# rand_weight取值自均匀分布[0,1)，shape=(20,20),requires_grad=False表示不计算梯度，故随机权重参数rand_weight其在训练期间作为常量，不会被反向传播更新
        self.linear=nn.Linear(20,20)
        
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)#torch.mm用于计算X, rand_weight的矩阵乘法，1应该是偏置
        X=self.linear(X)
        # 控制流
        while X.abs().sum()>1:#while循环：若X的L1范数大于1，将X按元素除以2
            X/=2
        return X.sum() #返回X中所有元素之和
    
net=FixedHiddenMLP()
net(X)

#block组合
class NestMLP(nn.Module):
    def __init__(self):
        super.__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                              nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
        
    def forward(self,X):
        return slef.linear(self.net(X))
    
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

## 2.参数管理

In [ ]:
import torch
from torch import nn

#单隐藏层的MLP
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8.1))
X=torch.rand(size=2,4)
net(X)

#2.1 参数访问
print(net[2].state_dict()) #通过参数的状态字典state_dict访问第2全连接层的参数

print(type(net[2].bias)) #访问第2层net的bias参数，打印其类型
print(net[2].bias) #打印net2的参数bias。参数是复合的对象，包含值、梯度和额外信息，不仅是值
print(net[2].bias.data) #打印bias参数值

net[2].weight.grad==None #未调用反向传播，参数的梯度处于初始状态

print(*[(name,param.shape) for name,param in net[0].named_parameters()]) #第一个全连接层的参数
print(*[(name,param.shape) for name,param in net.named_parameters()]) #所有层的参数。named_parameters()给出网络层的名字和参数的迭代器

net.state_dict()['2.bias'].data #访问网络层参数的另一种方式

#2.2 查看嵌套block的参数
def block1(): #定义模型block1
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                        nn.Linear(8,4),nn.ReLU())

def block2():#定义模型block2
    net=nn.Sequential() #net初始化
    for i in range(4): #添加block1到net
        net.add_module(f'block {i}',block1())
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1)) #rgnet[0]=block2,rgnet[1]=nn.Linear
rgnet(X)

print(rgnet) #查看网络各模块

rgnet[0][1][0].bias.data #访问第一个主要的块中、第二个子块的第一层的偏置项

#2.3 参数初始化
###2.3.1 使用内置初始化器--高斯随机变量
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01) #将所有权重参数初始化为标准差为0.01的高斯随机变量
        nn.init.zeros_(m.bias) #偏置参数设置为0
net.apply(init_normal) ##pytorch中的model.apply(fn)会递归地将函数fn应用到父模块的每个子模块submodule，也包括model这个父模块自身。
net[0].weight.data[0],net[0].bias.data[0] #输出net[0]的w,b初始值

###2.3.2 使用内置初始化器--给定常数
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1) #将所有权重参数初始化为常数1
        nn.init.zeros_(m.bias) #将偏置参数设置为0
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

###2.3.3 使用内置初始化器--不同block使用不同初始化方法
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight) #xavier初始化: 通过网络层时，输入和输出的方差相同，包括前向传播和后向传播
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
        
net[0].apply(xavier) #使用Xavier初始化方法初始化第1个神经网络层的weight
net[1].apply(init_42) #将第2个神经网络层weight初始化为常量值42
print(net[0].weight.data[0])
print(net[1].weight.data)

###2.3.4 自定义初始化
def my_init(m): #实现自定义参数函数
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *=m.weight.data.abs() >=5
        
net.apply(my_init) #net各层（共2层）应用自定义函数 my_init 初始化weight
net[0].weight[:2] #打印第1层net前2个weight（共8个）

###2.3.5 直接设置某个参数的值
net[0].weight.data[:]+=1 #第1层net的weight各元素值都+1
net[0].weight.data[0,0]=42 #第1层net的weight.data[0, 0]设置为42
net[0].weight.data[0] #打印第1层net第1个weight

###2.3.6 参数绑定
shared=nn.Linear(8,8) #给共享层一个名称，以便可以引用它的参数
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  shared,nn.ReLU(),
                  shared,nn.ReLU(),
                  nn.Linear(8,1)) #共6个net
net(X)

print(net[2].weight.data[0]==net[4].weight.data[0]) # 检查参数是否相同
net[2].weight.data[0,0]=100

print(net[2].weight.data[0]==net[4].weight.data[0])

"""
#参数绑定
1. 第2个隐藏层和第3个隐藏层参数绑定，其参数由相同的张量表示。改变其中一个层的参数，另一层的参数也会改变
2. 由于模型参数包含梯度，因此在反向传播期间，参数绑定的层的梯度会加到一起
"""

## 3 延后初始化

## 4 自定义层